In [1]:
%cd C:\Users\jigna\PredictiveAnalyticsProject

C:\Users\jigna\PredictiveAnalyticsProject


In [2]:
# 1 (1 point)

# Use pandas function read_csv to load data from the file 'GermanCredit.csv' into a dataframe named 'data'.
# The file can be downloaded from the Week 4 module in Brightspace.

import pandas as pd
data = pd.read_csv('data.csv')
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,38,technician,married,tertiary,no,127,yes,no,cellular,14,oct,113,1,50,2,success,no
1,41,housemaid,married,primary,no,365,no,no,cellular,8,aug,203,5,-1,0,unknown,no
2,39,management,single,tertiary,no,2454,yes,no,cellular,4,may,716,3,263,2,failure,yes
3,49,blue-collar,married,primary,no,6215,yes,no,cellular,11,may,549,1,-1,0,unknown,no
4,37,services,married,secondary,no,1694,yes,yes,cellular,29,jan,404,2,251,6,failure,no


In [3]:
# 2 (1 point)

# Write python code to prepare data for classification models in scikit-learn by dividing 
# it into train and test data sets. This model will be used to predict the value of Class
# based on other fields in the data set. Your y_train and y_test data sets should include
# the column Class from GermanCredit.csv and your X_train and X-test data sets should
# include all columns except the Class column.

from sklearn.model_selection import train_test_split

y = data['deposit']
X = pd.get_dummies(data.drop(columns=['deposit']))

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

C:\Users\jigna\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
X_train

,age,balance,day,duration,campaign,pdays,previous,job_admin,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
4175,28,762,18,115,3,-1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1
1108,31,50,29,395,1,-1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1232,36,3343,18,307,1,-1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3490,39,603,15,214,1,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7016,59,1265,7,849,1,-1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4373,55,538,7,353,2,-1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
7891,35,777,28,85,1,-1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
4859,21,423,8,104,5,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3264,29,1387,18,256,2,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [5]:
# 3 (1 point)

# Use a column transformer to update X_train so that all of its columns are standardized to have a 
# minimum value of zero and maximum value of one. Use the MinMaxScaler() function in scikit-learn to 
# do this. After transformation, X_train should be a dataframe containing the transformed data with 
# column names defined by the get_feature_names_out() function of the column transformer.

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

str_vars = [] # String variables
num_vars = [] # Numeric variables
for i in X.columns:
    if data.dtypes[i] == 'object':
        str_vars.append(i)
    else:
        num_vars.append(i)

ct = make_column_transformer(
    (MinMaxScaler(), num_vars),
    (OneHotEncoder(sparse=False), str_vars)
)

ct.fit(X_train)
X_train = pd.DataFrame(ct.transform(X_train), columns=ct.get_feature_names_out())

KeyError: 'job_admin'

In [65]:
# 4 (2 points)

# Use the transformed X_train dataset with k-fold cross-validation to train a Decision Tree classification
# model for several values of max_depth (1, 2, 3, 4, 5, 6, 7, 8, 9, 10). Which max_depth value gives the 
# best cross-validation score? What is the mean cross-validation score for the best Decision Tree 
# classification model based on this data set?

# Write your answers to the questions above as comments in the body of this cell.

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

for k in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    # Create linear regression object
    clsr = DecisionTreeClassifier(max_depth=k)

    # Define k-fold cross validatation
    kfold = StratifiedKFold(n_splits=5)
    
    #res = cross_validate(clsr, X_train, y_train, cv=kfold, return_train_score=True)
    #display(res)
    summary = cross_val_score(clsr, X_train, y_train, cv=kfold)
    print("Mean cross-validation scores with k = {}: {:.3f}".format(k, summary.mean()))
        
    ###########
    
    # max_depth value of 4 gives the best cross-validation score of 0.703

Mean cross-validation scores with k = 1: 0.715
Mean cross-validation scores with k = 2: 0.737
Mean cross-validation scores with k = 3: 0.778
Mean cross-validation scores with k = 4: 0.783
Mean cross-validation scores with k = 5: 0.794
Mean cross-validation scores with k = 6: 0.799
Mean cross-validation scores with k = 7: 0.808
Mean cross-validation scores with k = 8: 0.811
Mean cross-validation scores with k = 9: 0.809
Mean cross-validation scores with k = 10: 0.811


In [66]:
# 5 (2 points)

# Use the transformed X_train dataset with k-fold cross-validation to train a k-Nearest Neighbors 
# classification model for several values of k (1, 2, 3, 4, 5, 6, 7, 8, 9, 10). Which n_neighbors value gives the 
# best cross-validation score? What is the mean cross-validation score for the best k-NN classification model 
# based on this data set?

# Write your answers to the questions above as comments in the body of this cell.

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

for k in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    # Create linear regression object
    clsr = KNeighborsClassifier(n_neighbors=k)

    # Define k-fold cross validatation
    kfold = StratifiedKFold(n_splits=5)
    
    #res = cross_validate(clsr, X_train, y_train, cv=kfold, return_train_score=True)
    #display(res)
    summary = cross_val_score(clsr, X_train, y_train, cv=kfold)
    print("Mean cross-validation scores with k = {}: {:.3f}".format(k, summary.mean()))
        
    ###########
    
    # n_neighbors value of 5 gives the best cross-validation score of 0.729

Mean cross-validation scores with k = 1: 0.704
Mean cross-validation scores with k = 2: 0.694
Mean cross-validation scores with k = 3: 0.736
Mean cross-validation scores with k = 4: 0.728
Mean cross-validation scores with k = 5: 0.748
Mean cross-validation scores with k = 6: 0.740
Mean cross-validation scores with k = 7: 0.756
Mean cross-validation scores with k = 8: 0.748
Mean cross-validation scores with k = 9: 0.759
Mean cross-validation scores with k = 10: 0.756


In [67]:
# 6 (2 points)

# Use the transformed X_train dataset with k-fold cross-validation to train a Logistic Regression 
# classification model for several values of C (0.4, 0.6, 0.8, 1.0, 1.2). Which C value gives the 
# best cross-validation score? What is the mean cross-validation score for the best Logistic Regression
# classification model based on this data set?

# Write your answers to the questions above as comments in the body of this cell.

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

# Use cross-validation
for i in [0.4, 0.6, 0.8, 1.0, 1.2]:
    # Create linear regression object
    clsr = LogisticRegression(C=i, penalty='l1', solver='liblinear', max_iter=1000)

    # Define k-fold cross validatation
    kfold = StratifiedKFold(n_splits=5)
    
    #res = cross_validate(clsr, X_train, y_train, cv=kfold, return_train_score=True)
    #display(res)
    summary = cross_val_score(clsr, X_train, y_train, cv=kfold)
    print("Mean cross-validation scores with C = {}: {:.3f}".format(i, summary.mean()))
    
    ###########
    
    # C value of 0.8 gives the best cross-validation score of 0.767

Mean cross-validation scores with C = 0.4: 0.825
Mean cross-validation scores with C = 0.6: 0.825
Mean cross-validation scores with C = 0.8: 0.825
Mean cross-validation scores with C = 1.0: 0.825
Mean cross-validation scores with C = 1.2: 0.825


In [68]:
# 7 (2 points)

# From the results of questions 4, 5 and 6, choose the model that is most likely to generalize
# best on the X_test data set. Train the model using the complete X_train data and evaluate the
# model using the X_test data set.
    
###########

# I believe the Logistic Regression model is most likely to generalize best on the X_test set
    
###########
    
# Create logistic regression object
clsr = LogisticRegression(C=0.8, penalty='l1', solver='liblinear', max_iter=1000)

# Train the model using the training sets
clsr.fit(X_train, y_train)

print("Training set score: {:.3f}".format(clsr.score(X_train, y_train)))
print("Test set score: {:.3f}".format(clsr.score(X_test, y_test)))

Training set score: 0.829
Test set score: 0.815


In [ ]:
# 8 (1 point)

# Calculate predicted values of Class from the variables in X_test and
# display the confusion matrix calculated from the test data

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred = clsr.predict(X_test)
plt.rcParams['figure.figsize'] = (5, 5)
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred), display_labels=clsr.classes_[[0, 1]])
disp.plot(xticks_rotation=90)

In [ ]:
# 9 (1 point)

# Print the classification report based on the actual an predicted values 
# in the test data set

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

In [ ]:
# 10 (2 points)

# Comment on the effectiveness of this model for predicting Class in the GermanCredit data.
# What are the model's strengths and weaknesses? How does this model compare to a poor 
# baseline model that simply always predicts the value 'Good'?

# No code is required in this section, but if you use code to help yourself answer these questions,
# go ahead and include it here.

# Write your answers as comments in the body of this cell.

#######

# Going by the training and test scores of roughly 0.78 and 0.77 respectively, I would say the Logistic Regression model
# is moderately effective when it comes to predicting Class in the GermanCredit data. The model was similarly implemented 
# like the KNN and Decision Tree models but had better accuracy scores for this case. Compared to a model that always predicts 
# 'Good' (maybe right 50% of the time since it is only one of two options), the Logistic Regression model is accurate 77% of 
# the time making it a far better model.